step 1: Extract all concepts from training data

In [ ]:
with open("/content/train.txt","r") as f:
  lines = f.readlines()
  i = 0
  tmp = []
  entity = set()
  for i in range(len(lines)-1):
    line = lines[i].strip("\n")
    next_line = lines[i+1].strip("\n")
    if(line.endswith("B-Disease")):
      tmp = []
      if(next_line.endswith("I-Disease")):     # need to change I-GENE
        tmp.append(line.split(" ")[0])
        while(next_line.endswith("I-Disease")):
          tmp.append(next_line.split(" ")[0])
          i+=1
          next_line = lines[i+1].strip("\n")
        phrase = ""
        for j in range(len(tmp)-1):
          #wf.write(tmp[j] + " ")
          phrase += tmp[j]
          phrase += " "
        #wf.write(tmp[j+1] + "\n")
        phrase += tmp[j+1]
        #phrase += "\n"
        entity.add(phrase)
      else:
        entity.add(line.split(" ")[0])
        #wf.wirte(line + "\n")
print(entity)
with open("/content/train-disease-entities.txt","w") as wf:
  for ent in entity:
    print(ent)
    wf.write(ent + "\n")

Step 2: write the scripts for search for all related concepts



In [ ]:
!mkdir /content/related-concepts/

In [ ]:
!rm -r /content/related-concepts/

1. Each entity needs to create a file, and it's file name is the same as entity name.

2. Use the UMLS API, to creat the url for searching.

3. According to each url, get a json file.

4. Parse json file, get related concepts files corresponding to each entity, but only top 10 are enough.

5. Cui is saved as a dictionary for later use to look up parents and children.


In [ ]:
import requests
import json

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/search/current"

related_concepts_path = "/content/related-concepts/"

with open("/content/train-disease-entities.txt","r") as f:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n")
      url = api_url + "?apiKey=" + apikey + "&string=" + string
      response = requests.get(url)

      data = json.loads(response.content)

      with open(related_concepts_path + line.strip("\n") + ".txt","a") as wf:
        i = 1
        for concept in data['result']['results']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


Step 3: write the scripts for search for parents and children


In [ ]:
!mkdir /content/parents/

In [ ]:
!rm -r /content/parents/

1. Get the cuis of the given concepts



In [ ]:
import requests
import json

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/search/current"

related_concepts_path = "/content/related-concepts/"

with open("/content/train-disease-entities.txt","r") as f:
  with open("/content/train-disease-uis.txt","w") as wf:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n")
      url = api_url + "?apiKey=" + apikey + "&string=" + string + "&returnIdType=code&sabs=SNOMEDCT_US"
      response = requests.get(url)

      #print(string)
      data = json.loads(response.content)
      if(len(data['result']['results'])!=0):
        wf.write(string + "\t" + data['result']['results'][0]['ui']+"\n")
        #print(data['result']['results'])

2. Find its parents and children according to cuis. Only top 10 are enough.

In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

parents_children_path = "/content/parents/"

with open("/content/train-disease-uis.txt","r") as f:
  lines = f.readlines()
  for line in lines:
    #print(line)
    string = line.strip("\n").split("\t")[1]
    #print(string)
    url = api_url + string + "/parents?apiKey=" + apikey
    #print(url)
    response = requests.get(url)

    data = json.loads(response.content)

    with open(parents_children_path + line.strip("\n").split("\t")[0] + ".txt","a") as wf:
      i = 1
      for concept in data['result']:
        if(i<=10):
          #print(concept['name'])
          wf.write(concept['name']+"\n")
        i+=1

    url = api_url + string + "/children?apiKey=" + apikey
    response = requests.get(url)

    data = json.loads(response.content)
    #print(url,data)

    file_name = parents_children_path + line.strip("\n").split("\t")[0] + ".txt"
    if(os.path.exists(file_name)):
      with open(file_name,"r") as rf:
        lines = rf.readlines()
        i = len(lines)+1
        #print(file_name,i)
    else:
      i = 1
    with open(file_name,"a") as wf:
      #if(exists, i = lens; else: i = 0)
      if('result' in data):
        for concept in data['result']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


Step 4: write the scripts for search for siblings


In [ ]:
!mkdir /content/siblings/

In [ ]:
!rm -r /content/siblings/

1. Get the cuis of the parent of the given concept



In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

with open("/content/train-disease-uis.txt","r") as f:
  with open("/content/train-disease-parents-uis.txt","w") as wf:
    lines = f.readlines()
    for line in lines:
      #print(line)
      string = line.strip("\n").split("\t")[1]
      #print(string)
      url = api_url + string + "/parents?apiKey=" + apikey
      #print(url)
      response = requests.get(url)

      data = json.loads(response.content)

      if('result' in data):
        wf.write(line.strip("\n").split("\t")[0] + "\t" + data['result'][0]['ui']+"\n")

2. Find its children according to cuis. Only top 10 are enough.

In [ ]:
import requests
import json
import os

apikey = "7e7e253a-1168-4530-b8b4-24cb7daaa701"
api_url = "https://uts-ws.nlm.nih.gov/rest/content/current/source/SNOMEDCT_US/"

parents_children_path = "/content/siblings/"

with open("/content/train-disease-parents-uis.txt","r") as f:
  lines = f.readlines()
  for line in lines:
    #print(line)
    string = line.strip("\n").split("\t")[1]
    #print(string)
    url = api_url + string + "/children?apiKey=" + apikey
    response = requests.get(url)

    data = json.loads(response.content)
    #print(url,data)

    file_name = parents_children_path + line.strip("\n").split("\t")[0] + ".txt"
    i = 1
    with open(file_name,"a") as wf:
      if('result' in data):
        for concept in data['result']:
          if(i<=10):
            #print(concept['name'])
            wf.write(concept['name']+"\n")
          i+=1


Step 5: Replace given concepts with UMLS concepts -- process training data

1. Extract sentences and BIOs seperately from training data

In [ ]:
with open("/content/train-5-shot.txt","r") as f:
  with open("/content/train-sents.txt","a") as wf:
    with open("/content/train-BIOs.txt","a") as wf1:
      lines = f.readlines()
      for line in lines:
        if(line!="\n"):
          new_line = line.strip("\n").split(" ")
          wf.write(new_line[0] + " ")
          wf1.write(new_line[1] + " ")
        else:
          wf.write("\n")
          wf1.write("\n")

2.  Replace given concepts with UMLS concepts

In [ ]:
import os
import random
index = []

data_dir = "/content/parents/"
#data_dir = "/content/siblings/"
#data_dir = "/content/related_concepts/"
for file in os.listdir(data_dir):
  with open("/content/train-sents.txt","r") as f:
    with open(data_dir + file,"r") as f1:
      concept = file[:-4]
      #print(concept)
      with open("/content/train-5-shot.txt","a") as wf:
        lines = f.readlines()
        lines1 = f1.readlines()
        #print(len(lines1))
        count = -1
        for line in lines:
          count+=1
          if(concept in line):
            #print(count, line)
            new_line = line.strip("\n").split()
            start = int(line.find(concept))
            c = 0
            for i in range(start):
              if(line[i]==" "):
                c+=1                       # c: 第几个单词开始为concept
            lens = len(concept.split())
            #print(c,lens)                  # 从第c个单词开始，到 c+lens-1 的单词都需要我自己给BIO，其他的使用原本储存的BIO
            with open("/content/train-BIOs.txt","r") as f2:
              lines2 = f2.readlines()
              originalBIO = lines2[count]
              BIO_list = originalBIO.split(" ")
            if(len(lines1)>10):
              index = random.sample(range(0,len(lines1)),10)
            for l in range(len(lines1)):
              if(len(index)==0 or (len(index)!=0 and (l in index))):
                line1 = lines1[l]
                #print(line1)
                new_line1 = line1.strip("\n").split(" ")
                lens2 = len(new_line1)
                #print(lens2, new_line + "\n")
                for j in range(c):
                  wf.write(new_line[j] + " " + BIO_list[j] + "\n")
                for k in range(lens2):
                  if(k==0):
                    wf.write(new_line1[k] + " " + "B-Disease" + "\n")
                  else:
                    wf.write(new_line1[k] + " " + "I-Disease" + "\n")
                for j in range(c+lens, len(BIO_list)-1):
                  wf.write(new_line[j] + " " + BIO_list[j] + "\n")
                wf.write("\n")
            break